In [1]:
import requests
import pandas as pd
import numpy as np
import pickle
from numpy.random import random_sample

In [2]:
def save_pkl(file_dir, data):
    f = open(file_dir,"wb")
    pickle.dump(data, f, protocol=4)
    f.close()
    
def read_pkl(file_dir):
    f = open(file_dir,"rb")
    data = pickle.load(f)
    return data

In [3]:
# load the file
station_group = read_pkl("London/station_final.pkl")

In [4]:
station_group['ST5']

,station_id,PM25_Concentration,PM10_Concentration,NO2_Concentration,time_year,time_month,time_week,time_day,time_hour,temperature,...,max_humidity,max_wind_speed,var_PM25,var_PM10,var_NO2,var_temperature,var_pressure,var_humidity,var_wind_speed,var_wind_direction
time,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,ST5,25.0,29.0,14.5,2017.0,1.0,6.0,1.0,0.0,5.54,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 01:00:00,ST5,21.0,25.0,24.0,2017.0,1.0,6.0,1.0,1.0,5.58,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 02:00:00,ST5,20.0,23.0,28.1,2017.0,1.0,6.0,1.0,2.0,5.63,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 03:00:00,ST5,22.0,21.0,20.6,2017.0,1.0,6.0,1.0,3.0,5.68,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 04:00:00,ST5,22.0,26.0,34.6,2017.0,1.0,6.0,1.0,4.0,5.86,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 05:00:00,ST5,24.0,29.0,23.8,2017.0,1.0,6.0,1.0,5.0,6.05,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 06:00:00,ST5,16.0,23.0,17.2,2017.0,1.0,6.0,1.0,6.0,6.23,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 07:00:00,ST5,20.0,27.0,15.2,2017.0,1.0,6.0,1.0,7.0,6.49,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-01 08:00:00,ST5,20.0,25.0,20.6,2017.0,1.0,6.0,1.0,8.0,6.74,...,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [4]:
def flatten_Y(features, ratio = 1, stations = list(station_group.keys()), start = 14 * 24):  
    
    if(start == 14 * 24):
        length = int(9000 * ratio)
    else:
        length = 10806 - 72
    
    Y_PM25_list = []
    Y_PM10_list = []
    Y_NO2_list = []
    X_extend_list = []
    
    for name in station_group:
        if name in stations:
            # print(name)
            PM25_list = station_group[name]['PM25_Concentration'].values
            PM10_list = station_group[name]['PM10_Concentration'].values
            NO2_list = station_group[name]['NO2_Concentration'].values
            
            temperature_list = station_group[name]['temperature'].values
            pressure_list = station_group[name]['pressure'].values
            humidity_list = station_group[name]['humidity'].values
            wind_speed_list = station_group[name]['wind_speed'].values
            wind_direction_list = station_group[name]['wind_direction'].values
            
            original_PM25_list = []
            original_PM10_list = []
            original_NO2_list = []
            original_temperature_list = []
            original_pressure_list = []
            original_humidity_list = []
            original_wind_speed_list = []
            original_wind_direction_list = []
            
            X_this = station_group[name][features].values # features_matrix
            
            for i in range(start, length):
                Y_PM25_list.append(PM25_list[i+1 : i+49])
                Y_PM10_list.append(PM10_list[i+1 : i+49])
                Y_NO2_list.append(NO2_list[i+1 : i+49])
                
                differ_PM25_array = []
                differ_PM10_array = []
                differ_NO2_array = []
                differ_temperature_array = []
                differ_pressure_array = []
                differ_humidity_array = []
                differ_wind_speed_array = []
                differ_wind_direction_array = []
                
                original_PM25_array = PM25_list[i : i+48]
                original_PM10_array = PM10_list[i : i+48]
                original_NO2_array = NO2_list[i : i+48]
                original_temperature_array = temperature_list[i : i+72]
                original_pressure_array = pressure_list[i : i+72]
                original_humidity_array = humidity_list[i : i+72]
                original_wind_speed_array = wind_speed_list[i : i+72]
                original_wind_direction_array = wind_direction_list[i : i+72]
                
                for j in range(47):
                    differ_PM25_array.append(original_PM25_array[j+1] - original_PM25_array[j])
                    differ_PM10_array.append(original_PM10_array[j+1] - original_PM10_array[j])
                    differ_NO2_array.append(original_NO2_array[j+1] - original_NO2_array[j])
                    
                for j in range(71):    
                    differ_temperature_array.append(original_temperature_array[j+1] - original_temperature_array[j])
                    differ_pressure_array.append(original_pressure_array[j+1] - original_pressure_array[j])
                    differ_humidity_array.append(original_humidity_array[j+1] - original_humidity_array[j])
                    differ_wind_speed_array.append(original_wind_speed_array[j+1] - original_wind_speed_array[j])
                    differ_wind_direction_array.append(original_wind_direction_array[j+1] - original_wind_direction_array[j])
                
                this_PM25 = original_PM25_array[0]
                this_PM10 = original_PM10_array[0]
                this_NO2 = original_NO2_array[0]
                this_temperature = original_temperature_array[0]
                this_pressure = original_pressure_array[0]
                this_humidity = original_humidity_array[0]
                this_wind_speed = original_wind_speed_array[0]
                this_wind_direction = original_wind_direction_array[0]
                processed_PM25_list = []
                processed_PM10_list = []
                processed_NO2_list = []
                processed_temperature_list = []
                processed_pressure_list = []
                processed_humidity_list = []
                processed_wind_speed_list = []
                processed_wind_direction_list = []
                
                for j in range(47):
                    random_differ = (random_sample((1)) + 0.5)  * differ_PM25_array[j]
                    if(this_PM25 + random_differ >= 0):
                        this_PM25 = np.around(this_PM25 +  random_differ, decimals=1)
                    processed_PM25_list.append(this_PM25)
                    
                    random_differ = (random_sample((1)) + 0.5) * differ_PM10_array[j]
                    if(this_PM10 + random_differ >= 0):
                        this_PM10 = np.around(this_PM10 +  random_differ, decimals=1)
                    processed_PM10_list.append(this_PM10)
                    
                    random_differ = (random_sample((1)) + 0.5) * differ_NO2_array[j]
                    if(this_NO2 + random_differ >= 0):
                        this_NO2 = np.around(this_NO2 +  random_differ, decimals=1)
                    processed_NO2_list.append(this_NO2)
                    
                for j in range(71):
                    random_differ = ((random_sample((1)) - 0.5) / 5 + 1) * differ_temperature_array[j]
                    this_temperature = np.around(this_temperature +  random_differ, decimals=1)
                    processed_temperature_list.append(this_temperature)
                    
                    random_differ = ((random_sample((1)) - 0.5) / 5 + 1) * differ_pressure_array[j]
                    this_pressure = np.around(this_pressure +  random_differ, decimals=1)
                    processed_pressure_list.append(this_pressure)
                    
                    random_differ = ((random_sample((1)) - 0.5) / 5 + 1) * differ_humidity_array[j]
                    this_humidity = np.around(this_humidity +  random_differ, decimals=1)
                    processed_humidity_list.append(this_humidity)
                    
                    random_differ = ((random_sample((1)) - 0.5) / 5 + 1) * differ_wind_speed_array[j]
                    this_wind_speed = np.around(this_wind_speed +  random_differ, decimals=1)
                    processed_wind_speed_list.append(this_wind_speed)
                    
                    random_differ = ((random_sample((1)) - 0.5) / 5 + 1) * differ_wind_direction_array[j]
                    this_wind_direction = np.around(this_wind_direction +  random_differ, decimals=1)
                    processed_wind_direction_list.append(this_wind_direction)
                
                processed_PM25 = np.vstack(processed_PM25_list).flatten()
                processed_PM10 = np.vstack(processed_PM10_list).flatten()
                processed_NO2 = np.vstack(processed_NO2_list).flatten()
                processed_temperature = np.vstack(processed_temperature_list).flatten()
                processed_pressure = np.vstack(processed_pressure_list).flatten()
                processed_humidity = np.vstack(processed_humidity_list).flatten()
                processed_wind_speed = np.vstack(processed_wind_speed_list).flatten()
                processed_wind_direction = np.vstack(processed_wind_direction_list).flatten()
                
                historical_PM25_array = PM25_list[i-11 : i+1]
                historical_PM10_array = PM10_list[i-11 : i+1]
                historical_NO2_array = NO2_list[i-11 : i+1]
                historical_temperature_array = temperature_list[i-11 : i+1]
                historical_pressure_array = pressure_list[i-11 : i+1]
                historical_humidity_array = humidity_list[i-11 : i+1]
                historical_wind_speed_array = wind_speed_list[i-11 : i+1]
                historical_wind_direction_array = wind_direction_list[i-11 : i+1]
                
                PM25_array = np.hstack((historical_PM25_array, processed_PM25))
                PM10_array = np.hstack((historical_PM10_array, processed_PM10))
                NO2_array = np.hstack((historical_NO2_array, processed_NO2))
                temperature_array = np.hstack((historical_temperature_array, processed_temperature))
                pressure_array = np.hstack((historical_pressure_array, processed_pressure))
                humidity_array = np.hstack((historical_humidity_array, processed_humidity))
                wind_speed_array = np.hstack((historical_wind_speed_array, processed_wind_speed))
                wind_direction_array = np.hstack((historical_wind_direction_array, processed_wind_direction))
                
                for j in range(48):
                    # if you do not want to get the feature of historical PM2.5, you can Comment the following code
                    tmp = PM25_array[j:j+12]
                    
                    # if you do not want to get the feature of historical PM10, you can Comment the following code
                    tmp = np.hstack((tmp, PM10_array[j:j+12]))
                    
                    # if you do not want to get the feature of historical NO2, you can Comment the following code
                    tmp = np.hstack((tmp, NO2_array[j:j+12]))
                    
                    tmp = np.hstack((tmp, temperature_array[j:j+24]))
                    tmp = np.hstack((tmp, pressure_array[j:j+24]))
                    tmp = np.hstack((tmp, humidity_array[j:j+24]))
                    tmp = np.hstack((tmp, wind_speed_array[j:j+24]))
                    tmp = np.hstack((tmp, wind_direction_array[j:j+24]))
                    tmp = np.append(tmp, j)
                    X_extend_list.append(np.hstack((X_this[i, :], tmp)))
            
    Y_PM25 = np.vstack(Y_PM25_list)
    Y_PM10 = np.vstack(Y_PM10_list)
    Y_NO2 = np.vstack(Y_NO2_list)
    Y_PM25 = Y_PM25.flatten()
    Y_PM10 = Y_PM10.flatten()
    Y_NO2 = Y_NO2.flatten()
    X = np.vstack(X_extend_list)
    
    return X, Y_PM25, Y_PM10, Y_NO2    

In [5]:
def generate_train_data (features, k=1, name = 'all', start = 14 * 24):
    s = ['BL0','CD1','CD9','GN0','GN3','GR4','GR9','HV1','KF1','LW2','MY7','ST5','TH4']
    X, Y_PM25, Y_PM10, Y_NO2 = flatten_Y(features,ratio = k,stations = s, start = start)
    print('Generated data shape: ',X.shape, Y_PM25.shape,Y_PM10.shape,Y_NO2.shape)
    # save all of the X Y into .npy file
    np.save("London/X_"+name+".npy", X)
    np.save("London/Y_NO2_"+name+".npy", Y_NO2)
    np.save("London/Y_PM10_"+name+".npy", Y_PM10)
    np.save("London/Y_PM25_"+name+".npy", Y_PM25)
    print('Finish creating and saving the '+name+' version of X, Y_NO2, Y_PM10, and Y_PM25 data')

In [6]:
# If you want to removing some features in the generated X Y file (.npy), you can delete some of the following elements
# And [a,b] is differ from [b,a], which means you can control the feature order of the output data

features = ['PM25_Concentration','PM10_Concentration','NO2_Concentration', # feature 0-2
            'temperature','pressure','humidity','wind_speed', 'wind_direction', # feature 3-7
            'holiday','time_month','time_week','time_day','time_hour','station_index', # feature 8-13
            
            'mean_PM25','mean_PM10','mean_NO2','mean_temperature','mean_pressure','mean_humidity','mean_wind_speed','mean_wind_direction', # feature 14-21
            'max_PM25','max_PM10','max_NO2','max_temperature','max_pressure','max_humidity','max_wind_speed', # feature 22-28
            'var_PM25','var_PM10','var_NO2','var_temperature','var_pressure','var_humidity','var_wind_speed','var_wind_speed', # feature 29-36
            'pressure_temperature','temperature_humidity','PM','PM25_10' # feature 37-40
            
            # feature 41-52: historical PM2.5 features
            # feature 53-64: historical PM10 features
            # feature 65-76: historical NO2 features
            # feature 77-100: historical temperature and forecasting features
            # feature 101-124: historical pressure and forecasting features
            # feature 125-148: historical humidity and forecasting features
            # feature 149-172: historical wind_speed and forecasting features
            # feature 173-196: historical wind_direction and forecasting features
            
            # feature 197: prediction id
            ]

In [7]:
# Generate a testing dataset of X Y data (.npy)
# The generated file name is X_test.npy, etc.
generate_train_data(features, name = 'test', start = 9000)

Generated data shape:  (1082016, 198) (1082016,) (1082016,) (1082016,)
Finish creating and saving the test version of X, Y_NO2, Y_PM10, and Y_PM25 data


In [8]:
# Generate a small-sized version of X Y data (.npy)
# The generated file name is X_s.npy, etc.
generate_train_data(features, k = 0.05, name = 's')
# Reminder: k must be greater than 0.0374

Generated data shape:  (71136, 198) (71136,) (71136,) (71136,)
Finish creating and saving the s version of X, Y_NO2, Y_PM10, and Y_PM25 data


In [9]:
# You can also generate a small-sized version of X Y data (.npy) with removing some features.

In [10]:
# Generate a medium-sized version of X Y data (.npy)
# The generated file name is X_m.npy, etc.
generate_train_data(features, k = 0.2, name = 'm')

Generated data shape:  (913536, 198) (913536,) (913536,) (913536,)
Finish creating and saving the m version of X, Y_NO2, Y_PM10, and Y_PM25 data


In [ ]:
# You can also generate a medium-sized version of X Y data (.npy) with removing some features.

In [ ]:
# Generate a whole-sized version of X Y data (.npy)
# The generated file name is X_all.npy, etc.
generate_train_data(features, k = 1, name = 'all')

In [ ]:
# If you want to generate X Y files with other small size, you can change the ratio element
# For example, if you change the input ratio 'k' to 0.3 in the generate_train_data function, then you can get the X Y files with 30% of the whole size